In [13]:
from email import message
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
import pickle
import os.path
import base64
import email
from bs4 import BeautifulSoup
import csv 
import pandas as pd
SCOPES = ['https://www.googleapis.com/auth/gmail.readonly']

if os.path.exists('token.pickle'):
  
        # Read the token from the file and store it in the variable creds
        with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)

service = build('gmail', 'v1', credentials=creds)
query ='from:no-reply@thecitybank.com "transaction alert"'
result = service.users().messages().list(userId='me',q=query).execute()
messages = []
if 'messages' in result:
        messages.extend(result['messages'])
while 'nextPageToken' in result:
        page_token = result['nextPageToken']
        result = service.users().messages().list(userId='me',q=query, pageToken=page_token).execute()
        if 'messages' in result:
            messages.extend(result['messages'])

In [14]:
len(messages)

15

In [15]:
dfs = []
for msg in messages: 
    txt = service.users().messages().get(userId='me', id=msg['id']).execute()
    payload = txt['payload']
    if 'parts' in payload.keys():
                parts = payload.get('parts')[1]
                data = parts['body']['data']
                data = data.replace("-","+").replace("_","/")
                decoded_data = base64.b64decode(data)
                soup = BeautifulSoup(decoded_data , "html.parser")
                # body = soup.body()
                text = []
                for tr in soup.find_all('table')[1].find_all('table')[3].find_all('tr'):
                    for td in tr.find_all('td'):
                        text.append(str(td.string))
                keys_ = text[::2]
                values_ = text[1::2]
                dict_ = dict(zip(keys_,values_))
                temp_df = pd.DataFrame([dict_])
                dfs.append(temp_df)


                
# keys_ = text[::2]
# values_ = text[1::2]
# dict_ = dict(zip(keys_,values_))                

In [16]:
df = pd.concat(dfs).reset_index()

In [9]:
df[['Transaction Amount','Available Balance']].apply(lambda x:x.str.replace('BDT',''))

,Transaction Amount,Available Balance
0,5000.00,2771437.98
1,20000.00,2694228.82
2,10000.00,2753228.82
3,20000.00,2779295.82
4,20000.00,2799295.82
5,10000.00,2746708.17
6,20000.00,2848550.74
7,3000.00,2832904.07
8,3000.00,2832904.07
9,2500.00,2835904.07


In [17]:
df

,index,Particulars,Transaction Time & Date,Transaction Amount,Available Balance,Terminal Location,Card No.
0,0,None,23:05 06-AUG-2022,BDT 5000.00,BDT 2771437.98,ATM WDLShantinagar RATM>Shantinagar TSO BD,371599XXXXXXXXX
1,0,None,16:55 27-MAY-2022,BDT 20000.00,BDT 2694228.82,"ATM WDLShantinagar, Dhaka ATM>Shantinagar TSOBD",371599XXXXXXXXX
2,0,None,09:55 03-APR-2022,BDT 10000.00,BDT 2753228.82,"ATM WDLShantinagar, Dhaka ATM>Shantinagar TSOBD",371599XXXXXXXXX
3,0,None,19:20 28-MAR-2022,BDT 20000.00,BDT 2779295.82,"ATM WDLShantinagar, Dhaka ATM>Shantinagar TSOBD",371599XXXXXXXXX
4,0,None,19:19 28-MAR-2022,BDT 20000.00,BDT 2799295.82,"ATM WDLShantinagar, Dhaka ATM>Shantinagar TSOBD",371599XXXXXXXXX
5,0,None,12:48 05-MAR-2022,BDT 10000.00,BDT 2746708.17,"ATM WDLShanta Western Tower, Dh>Dhaka PolitecBD",371599XXXXXXXXX
6,0,None,11:44 05-FEB-2022,BDT 20000.00,BDT 2848550.74,ATM WDLTejgaon RATM>Tejgaon BD,371599XXXXXXXXX
7,0,None,10:31 04-JAN-2022,BDT 3000.00,BDT 2832904.07,"ATM WDLShanta Western Tower, Dh>Dhaka PolitecBD",371599XXXXXXXXX
8,0,None,10:31 04-JAN-2022,BDT 3000.00,BDT 2832904.07,"ATM WDLShanta Western Tower, Dh>Dhaka PolitecBD",371599XXXXXXXXX
9,0,None,10:30 04-JAN-2022,BDT 2500.00,BDT 2835904.07,"ATM WDLShanta Western Tower, Dh>Dhaka PolitecBD",371599XXXXXXXXX


In [10]:
df.to_csv('output_1.csv',index=False)

In [ ]:
import pandas as pd
dict_ = dict(zip(list(enumerate(keys_)),list(enumerate(values_))))

In [ ]:
pd.DataFrame.from_dict(dict_,orient='index')